<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/sergey_3d_random_synthetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:


!pip install ipdb

     |████████████████████████████████| 787 kB 6.7 MB/s 
     |████████████████████████████████| 370 kB 45.2 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=7aa169091b6113f0c1dbf3d3927ec137c830cd734b96ac87f067614f9ca836e3
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 

In [13]:
from scipy import *
from math import *
import numpy as np

def clean_float(fnumber):
    tol = 0.0000001
    if fnumber < tol and fnumber > -1.0 * tol:
        fnumber = 0.0
    return fnumber

def topology(frequency):
    vertex = 3
    for i in range(2,frequency+1):
        vertex += i + 1
    faces = frequency * frequency
    edges = (vertex + faces) - 1
    return [vertex, faces, edges]

def vlabels(vertex,faces,edges,frequency):
    labels = zeros([2,vertex + 1])
    points = zeros([3,vertex + 1])
    count = 1
    for i in range(0,frequency + 1):
        for j in range(0, i+1):
            labels[0,count] = i
            labels[1,count] = j
            points[0,count] = j
            points[1,count] = i - j
            points[2,count] = frequency - i
            count += 1
    return [points,labels]

def vcoords(points,labels,frequency):
    DEG_TO_RAD = .0174532925199433
    RAD_TO_DEG = 57.2957795130824    
    results = points.copy()
    results = results * 0.0
    for i in range(1,size(points,1)):
        X = points[0,i] * sin(72.0 * DEG_TO_RAD)
        Y = points[1,i] + (points[0,i] * cos(72.0 * DEG_TO_RAD))
        Z = frequency/2.0 + (points[2,i] / (2.0 * cos(36.0 * DEG_TO_RAD)))
        if (X == Y):
            results[0,i] = 0.0
        elif Y == 0:
            results[0,i] = 90.0
        else:
            results[0,i] = atan(X/Y) * RAD_TO_DEG
        # adjust value to correct quadrant
        if results[0,i] < 0.0:
            results[0,i] += 180
        # now calculate theta
        if (Z == 0):
            results[1,i] = 90.0
        else:
            results[1,i] = atan((pow(pow(X, 2.0) + pow(Y, 2.0), 0.5)) / Z) * RAD_TO_DEG
        # make sure the right quadrant is used
        if (results[1,i] < 0.0):
            results[1,i] += 180.0
        results[2,i] = 1
    return results

def oneface(labels,faceidx,vertex,count,i):
    faceidx[0,count] = i
    for j in range(i, vertex + 1):
        if ((labels[0,j] == labels[0,i] + 1) and (labels[1,j] == labels[1,i])):
            break
    faceidx[1,count] = j
    for k in range(i,vertex + 1):
        if ((labels[0,k] == labels[0,j]) and (labels[1,k] == labels[1,j] + 1)):
            break
    faceidx[2,count] = k
    return 0

def twoface(labels,faceidx,vertex,count,i):
    faceidx[0,count] = i
    for j in range(i, vertex + 1):
        if ((labels[0,j] == labels[0,i] + 1) and (labels[1,j] == labels[1,i] + 1)):
            break
    faceidx[1,count] = j
    for k in range(i,vertex + 1):
        if ((labels[0,k] == labels[0,j] - 1) and (labels[1,k] == labels[1,j])):
            break
    faceidx[2,count] = k
    return 0
            
def face_factor(vertex,faces,frequency,labels):
    faceidx = zeros([3,faces + 1])
    count = 1
    for i in range(1,vertex - (frequency + 1) + 1):
        if (labels[0,i] == 0 and labels[1,i] == 0):
            oneface(labels,faceidx,vertex,count,i)
            count += 1
        elif (labels[1,i] == 0):
            oneface(labels,faceidx,vertex,count,i)            
            count += 1             
            twoface(labels,faceidx,vertex,count,i)
            count += 1
        elif (labels[1,i] == labels[0,i]):
            oneface(labels,faceidx,vertex,count,i)            
            count += 1
        else:
            oneface(labels,faceidx,vertex,count,i)
            count += 1             
            twoface(labels,faceidx,vertex,count,i)
            count += 1             
    return  faceidx

def sphere2cart(vertices):
    DEG_TO_RAD = .0174532925199433    
    for i in range(1,size(vertices,1)):
        sX = clean_float(cos(vertices[0,i] * DEG_TO_RAD) \
                         * sin(vertices[1,i] * DEG_TO_RAD))
        sY = clean_float(sin(vertices[0,i] * DEG_TO_RAD) \
                         * sin(vertices[1,i] * DEG_TO_RAD))
        sZ = clean_float(cos(vertices[1,i] * DEG_TO_RAD))
        vertices[0,i] = sX
        vertices[1,i] = sY
        vertices[2,i] = sZ
    return 0

def points_facets(frequency):
    [v,f,e] = topology(frequency)
    [vp,lp] = vlabels(v,f,e,frequency)
    vc = vcoords(vp,lp,frequency)
    fidx = face_factor(v,f,frequency,lp)
    sphere2cart(vc)
    return [vc,fidx]

# returns spherical point coordinates
def spoints_facets(frequency):
    [v,f,e] = topology(frequency)
    [vp,lp] = vlabels(v,f,e,frequency)
    vc = vcoords(vp,lp,frequency)    
    fidx = face_factor(v,f,frequency,lp)
    return [vc,fidx]

def rotate_phi(phi, phi1, theta1, theta2):
    DEG_TO_RAD = .0174532925199433
    RAD_TO_DEG = 57.2957795130824        
    result = (sin(theta1 * DEG_TO_RAD)* sin((phi-phi1) * DEG_TO_RAD)) \
             / sin(theta2 * DEG_TO_RAD)
    if (result > 1.0):
        result = 1.0
    elif (result < -1.0):
        result = -1.0    
    return asin(result) * RAD_TO_DEG

def rotate_theta(phi, phi1, theta, theta1):
    DEG_TO_RAD = .0174532925199433
    RAD_TO_DEG = 57.2957795130824        
    result = cos(theta1 * DEG_TO_RAD) * cos(theta * DEG_TO_RAD) + \
             sin(theta1 * DEG_TO_RAD) * sin(theta * DEG_TO_RAD) * \
             cos((phi-phi1) * DEG_TO_RAD)
    if (result > 1.0):
        result = 1.0
    elif (result < -1.0):
        result = -1.0    
    return acos(result) * RAD_TO_DEG    
    
def icosa_sphere(facenum,points):
    RAD_TO_DEG = 57.2957795130824            
    sphere_pnt = points.copy() * 0.0
    if (facenum >= 0 and facenum <= 4):
        for i in range(1,size(points,1)):
            sphere_pnt[0,i] = points[0,i] + 72.0 * facenum
            sphere_pnt[1,i] = points[1,i]
            sphere_pnt[2,i] = 1.0
    elif (facenum >= 5 and facenum <= 9):
        k = facenum - 5		#phi rotation factor        
        for i in range(1,size(points,1)):        
            sphere_pnt[1,i] = rotate_theta(36.0, points[0,i], \
                                           (180.0 - (atan(2.0) * RAD_TO_DEG)), \
                                           points[1,i])
            sphere_pnt[0,i] = (rotate_phi(36.0, points[0,i], points[1,i], \
                                          sphere_pnt[1,i]) + 36.0) + 72.0 * k
            sphere_pnt[2,i] = 1.0
    elif (facenum >= 10 and facenum <= 14):
        k = facenum - 10		#phi rotation factor        
        for i in range(1,size(points,1)):
            sphere_pnt[1,i] = 180.0 - rotate_theta(36.0, points[0,i], \
                                                   (180.0 - (atan(2.0) * RAD_TO_DEG)), \
                                                   points[1,i])
            sphere_pnt[0,i] = (rotate_phi(36.0, points[0,i], points[1,i], \
                                          sphere_pnt[1,i]) + 36.0) + 72.0 * k + 36.0
            sphere_pnt[2,i] = 1.0
    else:
        k = facenum - 15		#phi rotation factor        
        for i in range(1,size(points,1)):            
            sphere_pnt[0,i] = (points[0,i] + 72.0 * k) + 36.0
            sphere_pnt[1,i] = 180.0 - points[1,i]
            sphere_pnt[2,i] = 1.0
    return sphere_pnt

def icosphere(maxlevel, R):
    [spoints,sfacets] = spoints_facets(maxlevel)
    sfacets = sfacets - 1
    pp = []
    for fnum in range(0,20):
        points = icosa_sphere(fnum,spoints)
        sphere2cart(points)
        points = points * R
        pp.append(points)
    return np.hstack(pp)

In [1]:
!pip install icosahedron

ERROR: Could not find a version that satisfies the requirement icosahedron (from versions: none)
ERROR: No matching distribution found for icosahedron


In [ ]:
from scipy.special import iv as besseli
import pylab as plt
import numpy as np
import ipdb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import icosahedron as ico

def vmf(mu, kappa, x):
    # single point function
    d = mu.shape[0]
    # compute in the log space
    logvmf = (d//2-1) * np.log(kappa) - np.log((2*np.pi)**(d/2)*besseli(d//2-1,kappa)) + kappa * np.dot(mu,x)
    return np.exp(logvmf)

def apply_vmf(x, mu, kappa, norm=1.0):
    delta = 1.0+vmf(mu, kappa, x)
    y = x * np.vstack([np.power(delta,3)]*x.shape[0])
    return y

plt.clf()

ax = plt.axes(projection ='3d')
numbumps = 1

w = np.random.rand(numbumps)
print('w',w.shape)
w = w/np.sum(w)
x = icosphere(30, 1.3) #np.random.randn(3,5000)
print('x',x.shape)

xnormed = x/np.linalg.norm(x, axis=0)
print('xnormed',xnormed.shape)

xx = xnormed.copy()*0
print('xx',xx.shape)

for i in range(numbumps):
    kappa = np.random.randint(1, 200)
    #print('kappa',kappa.shape)

    mu = np.random.randn(3); mu = mu/np.linalg.norm(mu)
    print('mu',mu.shape)

    y = apply_vmf(xnormed, mu, kappa)
    print('y',y.shape)

    xx += w[i]*y
    print('xx',xx.shape)

ax.scatter(xx[0,:], xx[1,:], xx[2,:], c=np.linalg.norm(xx,axis=0)-1, s=0.5, cmap=plt.cm.inferno)

#plt.gca().set_aspect(1)
#plt.axis('off')
plt.show()


In [20]:
xx.shape

(3, 9940)